# Import des librairies et chargement des données

In [2]:
import os

# grammaire abstraite de l'arbre syntaxique de Python
import ast

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.api.types import is_string_dtype, is_numeric_dtype

pd.options.mode.chained_assignment = None  # default='warn'

In [10]:
dossier_data = "data/"
nom_fichier = "batiments_data.csv"
batiments_data = pd.read_csv(dossier_data+nom_fichier)

In [11]:
pd.set_option("display.max_columns", None)
batiments_data.head()

,OSEBuildingID,BuildingType,PrimaryPropertyType,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFABuilding,ENERGYSTARScore,SiteEnergyUse,SteamUse,Electricity,NaturalGas,OtherFuelUse,GHGEmissions,BuildingAge,IsSteamUser,IsElectricityUser,IsNaturalGasUser,IsOtherFuelUser,NumberOfUsedEnergies,MostUsedEnergy,GatheredPrimaryPropertyType
0,3,NonResidential,Hotel,DOWNTOWN,1969,1.0,41.0,961990.0,18.0,73130656.0,19660404.0,49762435.0,3709900.0,0.0,2061.48,46,1.0,1.0,1.0,0.0,3.0,Electricity,Hotel
1,5,NonResidential,Hotel,DOWNTOWN,1926,1.0,10.0,61320.0,1.0,28229320.0,23458518.0,2769023.0,2001894.0,0.0,1936.34,89,1.0,1.0,1.0,0.0,3.0,Steam,Hotel
2,8,NonResidential,Hotel,DOWNTOWN,1980,1.0,18.0,107430.0,67.0,14829099.0,0.0,6066245.0,8763105.0,0.0,507.70,35,0.0,1.0,1.0,0.0,2.0,NaturalGas,Hotel
3,19,NonResidential,Hotel,DOWNTOWN,1922,1.0,11.0,67390.0,14.0,10711451.0,4403788.0,4089407.0,2218425.0,0.0,486.25,93,1.0,1.0,1.0,0.0,3.0,Steam,Hotel
4,25,NonResidential,Hotel,DOWNTOWN,1916,1.0,10.0,104352.0,83.0,7845112.0,3205497.0,1790665.0,2849024.0,0.0,411.22,99,1.0,1.0,1.0,0.0,3.0,Steam,Hotel


# Sélection des variables finales

In [12]:
batiments_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2322 entries, 0 to 2321
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   OSEBuildingID                2322 non-null   int64  
 1   BuildingType                 2322 non-null   object 
 2   PrimaryPropertyType          2322 non-null   object 
 3   Neighborhood                 2322 non-null   object 
 4   YearBuilt                    2322 non-null   int64  
 5   NumberofBuildings            2322 non-null   float64
 6   NumberofFloors               2322 non-null   float64
 7   PropertyGFABuilding          2322 non-null   float64
 8   ENERGYSTARScore              1532 non-null   float64
 9   SiteEnergyUse                2322 non-null   float64
 10  SteamUse                     2322 non-null   float64
 11  Electricity                  2322 non-null   float64
 12  NaturalGas                   2322 non-null   float64
 13  OtherFuelUse      

In [13]:
variables_ecartees = ["OSEBuildingID", "PrimaryPropertyType", "ENERGYSTARScore", "YearBuilt", "SteamUse", "Electricity",
                      "NaturalGas", "OtherFuelUse"]
batiments_data_modeles = batiments_data.drop(columns=variables_ecartees)

# Préparation des données

## Variables catégorielles

Pour les variables catégorielles, je vais toper chaque modalité.

In [14]:
batiments_data_modeles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2322 entries, 0 to 2321
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   BuildingType                 2322 non-null   object 
 1   Neighborhood                 2322 non-null   object 
 2   NumberofBuildings            2322 non-null   float64
 3   NumberofFloors               2322 non-null   float64
 4   PropertyGFABuilding          2322 non-null   float64
 5   SiteEnergyUse                2322 non-null   float64
 6   GHGEmissions                 2322 non-null   float64
 7   BuildingAge                  2322 non-null   int64  
 8   IsSteamUser                  2322 non-null   float64
 9   IsElectricityUser            2322 non-null   float64
 10  IsNaturalGasUser             2322 non-null   float64
 11  IsOtherFuelUser              2322 non-null   float64
 12  NumberOfUsedEnergies         2322 non-null   float64
 13  MostUsedEnergy    

In [22]:
s = (batiments_data_modeles.dtypes == 'object')
variables_categorielles = list(s[s].index)

In [55]:
modalites_topees = pd.get_dummies(batiments_data_modeles[variables_categorielles], drop_first=True)
batiments_data_topees = batiments_data_modeles.drop(columns=variables_categorielles)
batiments_data_topees = pd.concat([batiments_data_topees, modalites_topees], axis=1)
batiments_data_topees.rename(
    columns={"BuildingType_Nonresidential COS": "BuildingType_NonResidential_COS", 
             "BuildingType_Nonresidential WA": "BuildingType_NonResidential_WA", 
             "BuildingType_SPS-District K-12": "BuildingType_SPS_District_K-12", 
             "Neighborhood_GREATER DUWAMISH": "Neighborhood_GREATER_DUWAMISH", 
             "Neighborhood_LAKE UNION": "Neighborhood_LAKE_UNION", 
             "Neighborhood_MAGNOLIA / QUEEN ANNE": "Neighborhood_MAGNOLIA_QUEEN_ANNE", 
             "GatheredPrimaryPropertyType_Medical and Care": "GatheredPrimaryPropertyType_Medical_and_Care", 
             "GatheredPrimaryPropertyType_Mixed Use Property": "GatheredPrimaryPropertyType_Mixed_Use_Property", 
             "GatheredPrimaryPropertyType_Residence Hall": "GatheredPrimaryPropertyType_Residence_Hall", 
             "GatheredPrimaryPropertyType_Self-Storage Facility": "GatheredPrimaryPropertyType_Self_Storage_Facility", 
             "GatheredPrimaryPropertyType_Worship Facility": "GatheredPrimaryPropertyType_Worship_Facility"},
    inplace=True)

In [56]:
batiments_data_topees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2322 entries, 0 to 2321
Data columns (total 41 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   NumberofBuildings                                  2322 non-null   float64
 1   NumberofFloors                                     2322 non-null   float64
 2   PropertyGFABuilding                                2322 non-null   float64
 3   SiteEnergyUse                                      2322 non-null   float64
 4   GHGEmissions                                       2322 non-null   float64
 5   BuildingAge                                        2322 non-null   int64  
 6   IsSteamUser                                        2322 non-null   float64
 7   IsElectricityUser                                  2322 non-null   float64
 8   IsNaturalGasUser                                   2322 non-null   float64
 9   IsOtherF

In [57]:
batiments_data_topees.head()

,NumberofBuildings,NumberofFloors,PropertyGFABuilding,SiteEnergyUse,GHGEmissions,BuildingAge,IsSteamUser,IsElectricityUser,IsNaturalGasUser,IsOtherFuelUser,NumberOfUsedEnergies,BuildingType_NonResidential,BuildingType_NonResidential_COS,BuildingType_NonResidential_WA,BuildingType_SPS_District_K-12,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER_DUWAMISH,Neighborhood_LAKE_UNION,Neighborhood_MAGNOLIA_QUEEN_ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,MostUsedEnergy_NaturalGas,MostUsedEnergy_Steam,GatheredPrimaryPropertyType_Education,GatheredPrimaryPropertyType_Hotel,GatheredPrimaryPropertyType_Medical_and_Care,GatheredPrimaryPropertyType_Mixed_Use_Property,GatheredPrimaryPropertyType_Office,GatheredPrimaryPropertyType_Other,GatheredPrimaryPropertyType_Residence_Hall,GatheredPrimaryPropertyType_Restaurant,GatheredPrimaryPropertyType_Self_Storage_Facility,GatheredPrimaryPropertyType_Store,GatheredPrimaryPropertyType_Warehouse,GatheredPrimaryPropertyType_Worship_Facility
0,1.0,41.0,961990.0,73130656.0,2061.48,46,1.0,1.0,1.0,0.0,3.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,1.0,10.0,61320.0,28229320.0,1936.34,89,1.0,1.0,1.0,0.0,3.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2,1.0,18.0,107430.0,14829099.0,507.70,35,0.0,1.0,1.0,0.0,2.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,1.0,11.0,67390.0,10711451.0,486.25,93,1.0,1.0,1.0,0.0,3.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
4,1.0,10.0,104352.0,7845112.0,411.22,99,1.0,1.0,1.0,0.0,3.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0


## Standardisation des variables

Afin que mes variables aient le même ordre de grandeur, je vais standardiser les variables ayant les valeurs les plus élevées. À savoir *PropertyGFABuilding*, *SiteEnergyUse* et *GHGEmissions*.

In [58]:
batiments_data_topees.describe()

,NumberofBuildings,NumberofFloors,PropertyGFABuilding,SiteEnergyUse,GHGEmissions,BuildingAge,IsSteamUser,IsElectricityUser,IsNaturalGasUser,IsOtherFuelUser,NumberOfUsedEnergies,BuildingType_NonResidential,BuildingType_NonResidential_COS,BuildingType_NonResidential_WA,BuildingType_SPS_District_K-12,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER_DUWAMISH,Neighborhood_LAKE_UNION,Neighborhood_MAGNOLIA_QUEEN_ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,MostUsedEnergy_NaturalGas,MostUsedEnergy_Steam,GatheredPrimaryPropertyType_Education,GatheredPrimaryPropertyType_Hotel,GatheredPrimaryPropertyType_Medical_and_Care,GatheredPrimaryPropertyType_Mixed_Use_Property,GatheredPrimaryPropertyType_Office,GatheredPrimaryPropertyType_Other,GatheredPrimaryPropertyType_Residence_Hall,GatheredPrimaryPropertyType_Restaurant,GatheredPrimaryPropertyType_Self_Storage_Facility,GatheredPrimaryPropertyType_Store,GatheredPrimaryPropertyType_Warehouse,GatheredPrimaryPropertyType_Worship_Facility
count,2322.000000,2322.000000,2.322000e+03,2.322000e+03,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000
mean,1.179587,3.848407,1.042239e+05,8.442707e+06,194.385073,53.824289,0.067183,0.999569,0.726960,0.004307,1.798019,0.878122,0.054264,0.000431,0.049957,0.031438,0.028854,0.208010,0.069337,0.238587,0.089578,0.087855,0.037037,0.067614,0.049526,0.027993,0.022394,0.241602,0.013781,0.089147,0.038760,0.047373,0.066322,0.257537,0.154608,0.010336,0.006460,0.017227,0.074935,0.167959,0.036606
std,2.633764,5.858766,2.588143e+05,2.863574e+07,775.022987,32.421053,0.250393,0.020752,0.445617,0.065498,0.488679,0.327215,0.226586,0.020752,0.217903,0.174537,0.167433,0.405972,0.254081,0.426312,0.285638,0.283145,0.188893,0.251137,0.217011,0.164989,0.147995,0.428147,0.116607,0.285018,0.193063,0.212481,0.248898,0.437372,0.361609,0.101161,0.080131,0.130142,0.263344,0.373910,0.187834
min,1.000000,0.000000,3.636000e+03,1.144100e+04,0.080000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,2.854025e+04,1.201677e+06,19.392500,27.000000,0.000000,1.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,2.000000,4.799650e+04,2.414402e+06,48.855000,50.000000,0.000000,1.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,4.000000,9.485325e+04,6.685433e+06,134.772500,85.000000,0.000000,1.000000,1.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,111.000000,76.000000,9.320156e+06,8.739237e+08,16870.980000,116.000000,1.000000,1.00000

In [60]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False)
batiments_data_final = pd.DataFrame(scaler.fit_transform(batiments_data_topees))
batiments_data_final.columns = batiments_data_topees.columns

In [62]:
batiments_data_final.describe()

,NumberofBuildings,NumberofFloors,PropertyGFABuilding,SiteEnergyUse,GHGEmissions,BuildingAge,IsSteamUser,IsElectricityUser,IsNaturalGasUser,IsOtherFuelUser,NumberOfUsedEnergies,BuildingType_NonResidential,BuildingType_NonResidential_COS,BuildingType_NonResidential_WA,BuildingType_SPS_District_K-12,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER_DUWAMISH,Neighborhood_LAKE_UNION,Neighborhood_MAGNOLIA_QUEEN_ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,MostUsedEnergy_NaturalGas,MostUsedEnergy_Steam,GatheredPrimaryPropertyType_Education,GatheredPrimaryPropertyType_Hotel,GatheredPrimaryPropertyType_Medical_and_Care,GatheredPrimaryPropertyType_Mixed_Use_Property,GatheredPrimaryPropertyType_Office,GatheredPrimaryPropertyType_Other,GatheredPrimaryPropertyType_Residence_Hall,GatheredPrimaryPropertyType_Restaurant,GatheredPrimaryPropertyType_Self_Storage_Facility,GatheredPrimaryPropertyType_Store,GatheredPrimaryPropertyType_Warehouse,GatheredPrimaryPropertyType_Worship_Facility
count,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000
mean,0.447967,0.657005,0.402784,0.294895,0.250866,1.660522,0.268369,48.176758,1.631705,0.065767,3.680140,2.684203,0.239535,0.020757,0.229312,0.180163,0.172371,0.512487,0.272952,0.559775,0.313674,0.310350,0.196116,0.269290,0.228269,0.169704,0.151352,0.564419,0.118211,0.312846,0.200805,0.222999,0.266521,0.588955,0.427649,0.102195,0.080635,0.132395,0.284615,0.449292,0.194929
std,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215,1.000215
min,0.379767,0.000000,0.014052,0.000400,0.000103,0.030851,0.000000,0.000000,0.000000,0.000000,2.046775,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.379767,0.170721,0.110297,0.041973,0.025027,0.832972,0.000000,48.197515,0.000000,0.000000,4.093550,3.056753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.379767,0.341442,0.185488,0.084332,0.063050,1.542540,0.000000,48.197515,2.244561,0.000000,4.093550,3.056753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.379767,0.682885,0.366570,0.233515,0.173932,2.622318,0.000000,48.197515,2.244561,0.000000,4.093550,3.056753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.286878,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,42.154082,12.974810,36.018730,30.525205,21.773050,3.578693,3.994576,48.197515,2.244561,15.271029,8.187099,3.056753,4.414289,48.197515,4.590188,5.7

Toutes nos variables ont un écart-type quasiment égal à 1.

# Modélisation